In [1]:
# we want a read and write for the bulk data
# there will be different read and write functions for different data types

# in the end data will be read in based on the schema of the export file
    # but then put into the bulk data schema and with the normal schema

# writing 5 minute data files to the right location will be a part of it
# splitting the input data into the 5 minute segments will also be a part of it 


In [2]:
# so for the recent captures what would I do
# I'll have a dictionary of camera names and metadata for the bulk data location

# I'll go through a sorted list of the files
# I'll grab the device name from the folder name
# I'll go into the folder and combine all the parquets and write < ez jk I need these for the quereies
# for the bulk data, read it one file at a time and query the data for each 5 minute segment of the day
# this can use the same query function that could be used for querying the bulk data

# it will take in a bounds of times and retrun the frames between those times
# then a different function will take those frames and write them

# it does involve some way of linking the data to the time associated with the data
# let's just start with the video data I have 

# for every 5 minute segment
# search the deduplicated file names for the relevant files
# read the parquet and line up the times with the frame counts

# the metadata should have the time, but ngl it could only have the time
# we could get the frame offset by querieng the times for the 5 minute segment
# and then lining up the indexes

# in that case we can just write all the files and hope they line up
# although taking it file by file might help keep errors from propigating
# if I had good tests I could trust this would be less of an issue
# acctually if the timestamps line up there's no issues
# this is only a problem for the old capture data beofre I fixed that timestamp bug
# I will still write it file by file to make it so I only have to write one 

# this schema does mean that the files themselves will be of gratly varying length due to the timestamps
# as little as 20 frames per video in timelapse mode which would result in low compression
# this is why we had the heterogenious timesamp structure anyway

# the essence then could be the same thing, but we treat the file names as the indcators for where the data is
# then we query the timestampsdf for that videos frames and calculate offsets for the desired times
# return the frames that are 

# so the takeaway from this is that we can just cut off by day (which will still take some code for audio)
# but for the camera structure we don't need to do anything since it's already divided that way.

# so back to the standard way I was doing it where I just transfer all the videos as they are to the new day folder
# I can also just add all the timestamps to the metadata raw and just make a check incase there are too many timestamps



In [3]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import pandas as pd
import os
import sys
import subprocess
import cv2
import shutil

def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath + "dataImport/")
import rwWorkingTSDf
from rwWorkingTSDf import writeWorkingTSDf, readWorkingTSDF, fnString_to_dt, dt_to_fnString
recentCapPath = repoPath + "recentCaptures/"

In [4]:
# dictionaries
from collections import OrderedDict
fieldNames = ["responsiblePartyName", "instanceName", "developingPartyName", "deviceName", "dataType", "dataSource"]
myCams = {}
myCams["mobilepi"] = ["abhik", "mobilepi", "abhik", "piCam-raspberryPi5-Camv3120noir", "mp4", "piVidCap"]
myCams["bedroompi"] = ["abhik", "bedroompi", "abhik", "piCam-raspberryPi5-Camv3120", "mp4", "piVidCap"]
myCams["bathroompi"] = ["abhik", "bathroompi", "abhik", "piCam-raspberryPi5-Camv3noir", "mp4", "piVidCap"]
myCams["bathroomCam"] = ["abhik", "bathroomCam", "abhik", "pcCam-webcam", "mp4", "vidCap"]
myCams["deskCam"] = ["abhik", "deskCam", "abhik", "pcCam-webcam", "mp4", "vidCap"]
# right now we are going through and writing the intake code for recent video captures
# later I would like to refactor the pividcap and the normal vidcap
# and clear out everything un needed for video capture in the repo
# I honestly should be writing this in data import


# damn I have a bunch of files seperated by the current timezone day and not the UTC day
# this does mean that there will be some cutting together and apart of the videos

In [5]:
def geBulkLocation(time, responsiblePartyName, instanceName, developingPartyName, deviceName, dataType, dataSource):
    dataFolderName = "_".join([responsiblePartyName, instanceName, developingPartyName, deviceName, dataType, dataSource]) + "/"
    dataFolderName += time.strftime("%Y") + "/" + time.strftime("%m-%d") + "/"
    return repoPath + "bulkData/" + dataFolderName

In [6]:
def frame_to_time(frame_index, fps):
    """Convert frame index to timestamp format HH:MM:SS.milliseconds"""
    seconds = frame_index / fps
    return f"{int(seconds // 3600):02}:{int((seconds % 3600) // 60):02}:{seconds % 60:.3f}"

def splitVidffmpeg(input_file_location, boundary_indexes, output_paths):
    print(input_file_location)
    cap = cv2.VideoCapture(input_file_location)
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(fps)
    cap.release()

    for i, (start_frame, end_frame) in enumerate(boundary_indexes):
        start_time = frame_to_time(start_frame, fps)
        end_time = frame_to_time(end_frame, fps)
        
        command = [
            "ffmpeg", "-y", "-i", input_file_location,
            "-ss", start_time, "-to", end_time,
            "-c", "copy", output_paths[i]
        ]
        print(command)
        subprocess.run(command)

def splitVidopencv(input_file_location, boundary_indexes, output_paths):
    cap = cv2.VideoCapture(input_file_location)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'avc1')
    
    for i, (start_frame, end_frame) in enumerate(boundary_indexes):
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        out = cv2.VideoWriter(output_paths[i], fourcc, fps, (width, height))

        for frame_idx in range(start_frame, end_frame):
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)

        out.release()
        print(f"Saved: {output_paths[i]}")

    cap.release()


def fnString_to_dt(fn_string):
    fn_string = fn_string[:17] + ',' + fn_string[18:]
    return datetime.fromisoformat(fn_string.replace(",", "."))



In [7]:
# read in the metadata parquet
workingdfs = {}
for k in myCams.keys():
    workingdfs[k] = readWorkingTSDF(*myCams[k])


Empty DataFrame
Columns: []
Index: [2024-12-21 06:54:15.013774+00:00, 2024-12-21 06:54:15.103128+00:00, 2024-12-21 06:54:15.200101+00:00, 2024-12-21 06:54:15.300111+00:00, 2024-12-21 06:54:15.400119+00:00]
read in 424385 rows from 28 files, retruning 424385 rows
Empty DataFrame
Columns: []
Index: [2024-12-26 07:09:45.016039+00:00, 2024-12-26 07:09:45.100187+00:00, 2024-12-26 07:09:45.200104+00:00, 2024-12-26 07:09:45.300101+00:00, 2024-12-26 07:09:45.400087+00:00]
read in 426197 rows from 25 files, retruning 426197 rows
Empty DataFrame
Columns: []
Index: [2024-12-26 17:47:00.002571+00:00, 2024-12-26 17:47:15.000189+00:00, 2024-12-26 17:47:30.000140+00:00, 2024-12-26 17:47:45.000151+00:00, 2024-12-26 17:48:00.000295+00:00]
read in 407268 rows from 3 files, retruning 407268 rows
Empty DataFrame
Columns: []
Index: [2024-11-15 01:19:15.001407+00:00, 2024-11-15 01:19:15.100091+00:00, 2024-11-15 01:19:15.200086+00:00, 2024-11-15 01:19:15.300096+00:00, 2024-11-15 01:19:15.400095+00:00]
read i

In [ ]:



foldersToImport = sorted(os.listdir(recentCapPath))
for f in foldersToImport:
    camName = f.split("_")[0] ################################################ change all of these once we update the vid cap
    folderPath = recentCapPath + f + "/"
    # get a list of file name bases (withouth the file types)
    fileNameBases = sorted(list(set([x.split(".")[0] for x in os.listdir(folderPath)])))
    
    # combine all parquets
    for i, fileName in enumerate(fileNameBases):
        source = folderPath + fileName + ".parquet"
        rDf = pd.read_parquet(source)
        rDf.index = rDf.index.tz_convert('UTC')
        if i == 0:
            readDf = rDf
        else:
            readDf = pd.concat([readDf, rDf])

    # add to the full metadata parquet
    if type(workingdfs[camName]) == type(None):
        workingdfs[camName] = readDf
    else:
        workingdfs[camName] = pd.concat([workingdfs[camName], readDf]).sort_index()

    # for each file in a folder
    for fileName in fileNameBases:
        startTime = fnString_to_dt(fileName.split("_")[-2]).astimezone(ZoneInfo("UTC"))
        endTime = fnString_to_dt(fileName.split("_")[-1]).astimezone(ZoneInfo("UTC"))
        # if not a boundary file rename, move and contuinue
        #print(startTime.day)
        #print(endTime.day)
        if startTime.day == endTime.day:
            # generate new file name
            newFileName = dt_to_fnString(startTime) + "_" + dt_to_fnString(endTime) + ".mp4"
            source = folderPath + fileName + ".mp4"
            destination = geBulkLocation(startTime, *myCams[camName])
            # move to the right location
            #print(destination)
            #print(os.path.exists(destination))
            if not os.path.exists(destination):
                print("made directory " + destination)
                os.makedirs(destination)
            shutil.copy2(source, destination  + newFileName)
            continue

        # the file contains more than one day of data
        frameTimes = workingdfs[camName][startTime:endTime].copy()
        #print(frameTimes.head().index)
        frameTimes.index = pd.to_datetime(frameTimes.index, utc=True)
        frameTimes['date'] = frameTimes.index.normalize()
        boundaries = frameTimes.groupby('date').apply(lambda x: (x.index[0], x.index[-1]), include_groups=False)
        index_boundaries = [(frameTimes.index.get_loc(start), frameTimes.index.get_loc(end))
                                for day, (start, end) in boundaries.items()]
        
        output_paths = [geBulkLocation(x[0], *myCams[camName]) + 
                        dt_to_fnString(x[0]) + "_" + dt_to_fnString(x[1]) + ".mp4" 
                        for x in boundaries]

        splitVidopencv(folderPath + fileName  + ".mp4", index_boundaries, output_paths)

    # delete the folder
    #os.rmdir(recentCapPath + f)




DatetimeIndex(['2025-02-10 23:20:43.400067+00:00',
               '2025-02-10 23:20:43.500068+00:00',
               '2025-02-10 23:20:43.600497+00:00',
               '2025-02-10 23:20:43.700067+00:00',
               '2025-02-10 23:20:43.800062+00:00'],
              dtype='datetime64[ns, UTC]', name='sampleDT', freq=None)
Saved: /home/chowder/Documents/bulkData/abhik_bathroompi_abhik_piCam-raspberryPi5-Camv3noir_mp4_piVidCap/2025/02-10/2025-02-10T232043,400067+0000_2025-02-10T235945,000224+0000.mp4
Saved: /home/chowder/Documents/bulkData/abhik_bathroompi_abhik_piCam-raspberryPi5-Camv3noir_mp4_piVidCap/2025/02-11/2025-02-11T000000,000235+0000_2025-02-11T032057,000063+0000.mp4
DatetimeIndex(['2025-02-11 22:42:05.600175+00:00',
               '2025-02-11 22:42:05.700089+00:00',
               '2025-02-11 22:42:05.800182+00:00',
               '2025-02-11 22:42:05.900081+00:00',
               '2025-02-11 22:42:06.000083+00:00'],
              dtype='datetime64[ns, UTC]', name='sampleDT

In [9]:
print(type(frameTimes.head(1).index))

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


In [10]:
pd.to_datetime(frameTimes.index)

DatetimeIndex(['2025-02-19 20:42:00.000347+00:00',
               '2025-02-19 20:42:15.000127+00:00',
               '2025-02-19 20:42:30.000238+00:00',
               '2025-02-19 20:42:45.000158+00:00',
               '2025-02-19 20:43:00.000164+00:00',
               '2025-02-19 20:43:15.000151+00:00',
               '2025-02-19 20:43:30.000203+00:00',
               '2025-02-19 20:43:45.000240+00:00',
               '2025-02-19 20:44:00.000157+00:00',
               '2025-02-19 20:44:15.000157+00:00',
               ...
               '2025-02-20 04:09:30.000161+00:00',
               '2025-02-20 04:09:45.000232+00:00',
               '2025-02-20 04:10:00.000156+00:00',
               '2025-02-20 04:10:15.000154+00:00',
               '2025-02-20 04:10:30.000158+00:00',
               '2025-02-20 04:10:45.000241+00:00',
               '2025-02-20 04:11:00.000253+00:00',
               '2025-02-20 04:11:15.000231+00:00',
               '2025-02-20 04:11:30.000150+00:00',
            

In [11]:
for k in myCams.keys():
    writeWorkingTSDf(*myCams[k], workingdfs[k])

hashes match for 2024-12-21T065415,013774+0000_2024-12-27T212238,607328+0000_98f24635_.parquet.gzip
the hashes don't match for start time 2024-12-27 21:22:38.607328+00:00 to end time 2024-12-30 11:00:30.000163+00:00
reading and combining to see if there's any new data
hashes match now
nothing to add
nothing to add
nothing to add
nothing to add
nothing to add
nothing to add
nothing to add
nothing to add
nothing to add
nothing to add
nothing to add
nothing to add
nothing to add
nothing to add
nothing to add
nothing to add
nothing to add
nothing to add
nothing to add
nothing to add
nothing to add
the hashes don't match for start time 2025-02-09 14:13:27.900069+00:00 to end time 2025-02-11 04:06:28.000071+00:00
reading and combining to see if there's any new data
hashes match now
hashes match for 2025-02-11T040628,000071+0000_2025-02-12T175112,600067+0000_fd2f6523_.parquet.gzip
hashes match for 2025-02-12T175112,600067+0000_2025-02-14T093013,200071+0000_2d50fd59_.parquet.gzip
hashes match 

In [12]:
import importlib
importlib.reload(rwWorkingTSDf)
import rwWorkingTSDf
from rwWorkingTSDf import writeWorkingTSDf, readWorkingTSDF, fnString_to_dt, dt_to_fnString